### To explore various methods of reducing the issues brought upon by imbalanced datasets in ML problems

Learning phase and subsequent prediction of models can be affected by the problem of imbalanced datasets. More often than not, the decision function of classifiers would usually favour the majority class.

- Undersampling
- Oversampling
- Generate synthetic data for minority class

How should we rebalance dataset:
- Equal proportion of both classes?
- Majority stays represented?

No straight-forward answer as modifying datasets with resampling methods is changing reality.
Exercise with caution.

Evaluation methods

Accuracy based: classifier would as a matter of fact, always predict the majority class if real data is imbalanced.
Cost-based: 

In [ ]:
# Loading in datsets


In [ ]:
# Illustrate imbalance dataset